In [7]:
import socket
import ssl
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
import os


In [8]:

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = private_key.exchange(public_key)
    return shared_key

def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)
        )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP( mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None))
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.load_verify_locations(cafile="server.crt")
    context.check_hostname = False
    client_socket.connect(("localhost", 8443))
    print("Connected to server.")
    
    with context.wrap_socket(client_socket, server_hostname="localhost") as ssock:
        
        print("SSL/TLS connection established.")

        encryption_choice = input("Please choose encryption method: 1 for RSA or 2 for AES (Diffie-Hellman)")
        # ssock.send(.encode())
        ssock.send(encryption_choice.encode())
        # encryption_choice = ssock.recv(1024).decode()
        # print(f"Encryption choice: {encryption_choice}")
    
        if encryption_choice == "1":
            # RSA encryption and PKI key exchange
            private_key = generate_rsa_keypair()
            public_key = private_key.public_key()
            print(f"Public key generated. {public_key}")
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            
            #receive public key from server
            received_public_key = ssock.recv(2048)
            server_public_key = load_pem_public_key(received_public_key, backend=default_backend())
            
            while True:
                #send msg to server
                msg_for_server = input("Enter your message for server: ")
                #encrypt msg with server's public key
                e_msg_for_server = rsa_encrypt(server_public_key, msg_for_server)
                print(f"Encrypted RSA message for server: {e_msg_for_server}")
                #send encrypted msg to server
                ssock.send(e_msg_for_server)
                print("Message sent to server.")
                
                #-----------------------------------------------
                #receive msg from server
                e_msg_from_server = ssock.recv(2048)
                d_msg_from_server = rsa_decrypt(private_key, e_msg_from_server)
                print(f"Decrypted RSA message from server: {d_msg_from_server}")
                if d_msg_from_server == "exit":
                    break

        elif encryption_choice == "2":
            # AES encryption and Diffie-Hellman key exchange
            bob_private_key, bob_public_key = generate_dh_keypair()
            ssock.send(bob_public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))

            received_public_key = ssock.recv(2048)
            received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
            shared_key = derive_dh_shared_key(bob_private_key, received_public_key)

            data = ssock.recv(1024)
            message = aes_decrypt(shared_key, data)
            print(f"Decrypted AES message: {message}")
    



In [9]:
if __name__ == "__main__":
    start_client()
#openssl req -x509 -newkey rsa:4096 -keyout ca.key -out ca.crt -days 365
    """
    info for ca.crt
    PEM pass phrase: 03455
    country name: PK
    state or province name: Punjab
    locality name: Rawalpindi
    organization name: FAST
    organizational unit name: Computing
    common name: FARQ
    email address: i200621@nu.edu.pk
    """



Connected to server.
SSL/TLS connection established.
Public key generated. <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x000001F4F41E4070>
Encrypted RSA message for server: b'\xae\xc3\xe9\x98\x9c\x92\x97\r\x9e\x8d\xc5\xc1\xf3\xb9j\xd1\x01\xf7\xed\xd0A7\xc4\xb6\xd0\x96@\x14\x90ma\xe9\x86_\xd4\xb2}\xfcS\x93\x8a-PDR\xfbK\xde\xa5\x8a\xc1\xcd\xcb\r\x19B\x1c\xee\xcd&\x8c[\xc7\x11\xf6\xde~\xfc\xc4\x1cc\xf2\xa9\xce\xed\xa8\x13\x82\xfa\xf0\xfd,\xdf\xf8\xf7\xea\x13\x8c*\x1f\x82\xc6\x8c}4\xc0\x9c\x1e\x07\xa1\x0f\x1a3\x18#|\xe7.:\xddY\x87\xe7C\x84\xbf\xf1\xd5\xeb\xd3:AX[D6\x93F0!\r\xd4\x80\xf8\x1d\xc3\xeba"\x07\x07Oh\xf0\x9e\x9f\xdc\x88P\xcd\xe3\xc4\xe7\xe2\x0c\xa5\xf4\xd6\x8fr\xda\xb7(\xed\x1dje\x02\xacQ\x86\xa7\xcf\xc0\x05\x00\x8a\xcf\x02\x9c^d1\r\xf9Vu\xebe\xe8\xaa\xb7\xf8\xe6\x1e\xdb\xd6/\xdc\xf6\xf4G\xa2\x8f\xc6c\x11\x85\xb9\xef\xdc6\xdc+\xb3\xd8~\'\xbdx\xc0\xab:\xf5\xee\x10\x07\x01F\x16R\x0b\xe5\xca\t\xd0\xc8S\x02\xe4\x87\xb3IP\x13\xa4\xd9\x96\x96\x9d\xcf\xbe\xf2\xcaJ\x